In [66]:
import numpy
from urllib.request import urlopen
from collections import defaultdict
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import matplotlib.pyplot as plt
import scipy
import scipy.optimize
import random
import copy
import gzip
import json
import string

def parseData(fname):
    for l in open(fname):
        yield eval(l)
def feature_label(data):
    return data["fit"]
def feature_X(data):
    return [1,data['height'],data['weight'],int(data['rating'])]
def str_to_inch(s):
    s = s.replace('\"', '')
    height=s.split("\'")
    return int(height[0])*12+int(height[1])
def str_to_weight(s):
    s = s.replace('lbs', '')
    return int(s)




test_data = list(parseData("modcloth_final_data.json"))
#random.shuffle(data)
print(test_data[:10])




[{'item_id': '123373', 'waist': '29', 'size': 7, 'quality': 5, 'cup size': 'd', 'hips': '38', 'bra size': '34', 'category': 'new', 'bust': '36', 'height': '5ft 6in', 'user_name': 'Emily', 'length': 'just right', 'fit': 'small', 'user_id': '991571'}, {'item_id': '123373', 'waist': '31', 'size': 13, 'quality': 3, 'cup size': 'b', 'hips': '30', 'bra size': '36', 'category': 'new', 'length': 'just right', 'height': '5ft 2in', 'user_name': 'sydneybraden2001', 'fit': 'small', 'user_id': '587883'}, {'item_id': '123373', 'waist': '30', 'size': 7, 'quality': 2, 'cup size': 'b', 'shoe size': '9.00', 'bra size': '32', 'category': 'new', 'length': 'slightly long', 'height': '5ft 7in', 'user_name': 'Ugggh', 'fit': 'small', 'user_id': '395665'}, {'item_id': '123373', 'category': 'new', 'size': 21, 'quality': 5, 'user_name': 'alexmeyer626', 'length': 'just right', 'fit': 'fit', 'cup size': 'dd/e', 'user_id': '875643'}, {'item_id': '123373', 'category': 'new', 'size': 18, 'quality': 5, 'user_name': 'd

In [67]:
def TruePos(pred,y):
    res=0
    for pred_, y_ in zip(pred, y):
        if pred_ and y_:
            
            res+=1
    return res

def FalsePos(pred,y):
    res=0
    for pred_, y_ in zip(pred, y):
        if pred_ and not y_:
            
            res+=1
    return res

def TrueNeg(pred,y):
    res=0
    for pred_, y_ in zip(pred, y):
        if not pred_ and not y_:
            
                res+=1
    return res

def FalseNeg(pred,y):
    res=0
    for pred_, y_ in zip(pred, y):
        if not pred_ and y_:
            
            res+=1
    return res
def BER(pred,train_y):
    TP=TruePos(pred,train_y)
    FP=FalsePos(pred,train_y)
    TN=TrueNeg(pred,train_y)
    FN=FalseNeg(pred,train_y)
    TPR= TP/(TP+FN)
    TNR= TN/(TN+FP)
    FPR= FP/(FP+TN)
    FNR=FN/(FN+TP)
    #BER= 1-0.5*(TPR+TNR)
    BER= 0.5*(FPR+FNR)
    print("TPR for training set is :", TPR)
    print("TNR for training set is :", TNR)
    print("FPR for training set is :", FPR)
    print("FNR for training set is :", FNR)
    print("BER for training set is :", BER)


In [2]:
#f = open('renttherunway_final_data.json', "r")
data = list([json.loads(line) for line in open('renttherunway_final_data.json', 'r')])
    
data1=[]
data_test=[]
for i in data:
    if 'weight' in i and 'bust size' in i and 'height' in i and 'rating' in i and 'review_text' in i:
        if i['rating'] != None:
            i['weight']=str_to_weight(i['weight'])
            i['height']=str_to_inch(i['height'])
            data1.append(i)

for i in test_data:
    if 'weight' in i and 'bust size' in i and 'height' in i and 'rating' in i and 'review_text' in i:
        i['weight']=str_to_weight(i['weight'])
        i['height']=str_to_inch(i['height'])
        data_test.append(i)

In [3]:
#print(len(data_train),len(data))
print(len(data_test),len(test_data))    #different data sets cannot be test with training set

random.shuffle(data1)
data_train=data1[:len(data1)//10*9]
data_test=data1[len(data1)//10*9:]
print(len(data_train),len(data_test),len(data1))

0 82790
136152 15132 151284


In [4]:
print(type(data[0]))
for key in data[0]:
    print(key,':',data[0][key])



string1= "5'8\""
#print(data_train[1]['height'])
#print(str_to_inch(data_train[1]['height']))
    

<class 'dict'>
fit : fit
user_id : 420272
bust size : 34d
item_id : 2260466
weight : 137
rating : 10
rented for : vacation
review_text : An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.
body type : hourglass
review_summary : So many compliments!
category : romper
height : 68
size : 14
age : 28
review_date : April 20, 2016


In [5]:
#print(test_data[:1])
#l=[feature_height(d) for d in data_train]
dic= defaultdict(int)
#print(l[0])
#for i in l:                                       #num of fit/small/large
#    j=str(i)
#    dic[j]+=1

p=[d['category'] for d in data_train]             #num of items per category
#for i in p:
#    j=str(i)
#    dic[j]+=1
    
print(dic)

defaultdict(<class 'int'>, {})


In [6]:
#possible missing values:

#bust size
#weight
#rating
#review_text
#height


In [7]:
usersPeritem = defaultdict(set)
itemPerUser = defaultdict(set)
for d in data:
    user,item = d['user_id'], d['item_id']
    usersPeritem[item].add(user)
    itemPerUser[user].add(item)
    






In [8]:
#for i in itemPerUser:
   # print(len(itemPerUser[i]))
#print(len(itemPerUser))          #

In [9]:
X_train=[feature_X(d) for d in data_train]
X_test=[feature_X(d) for d in data_test]

Y_train=[feature_label(d) for d in data_train]
Y_test=[feature_label(d) for d in data_test]

In [10]:
mod = linear_model.LogisticRegression(C=2.0,)
mod.fit(X_train, Y_train)




LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
pred_valid= mod.predict(X_test)
correct1 = pred_valid==Y_test

print("accuracy feature:weight,height,rating", sum(correct1)/len(correct1))
#print(len(correct1))
print('done..')

accuracy feature:weight,height,rating 0.7380385937086968
done..


In [12]:


dic1= defaultdict(int)
dic2=defaultdict(int)

#print(l[0])
for i in pred_valid:                                       #num of fit/small/large
    j=str(i)
    dic1[j]+=1
for i in Y_test:                                       #num of fit/small/large
    j=str(i)
    dic2[j]+=1
    
print(dic1,dic2)

defaultdict(<class 'int'>, {'fit': 14858, 'small': 274}) defaultdict(<class 'int'>, {'fit': 11163, 'small': 2005, 'large': 1964})


In [13]:
print(type(Y_train[1]))

<class 'str'>


In [14]:
print([feature_label(d) for d in data_train[:5]])

['fit', 'fit', 'fit', 'small', 'small']


In [15]:
#try bag of words methods
sp = set(string.punctuation)
wordCount = defaultdict(int)
for d in data1:
    r = ''.join([c for c in d['review_text'].lower() if not c in sp])
    tokens = r.split()
    for w in tokens:
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()



In [ ]:
#SET UP CATEGORY FOR data
    

In [16]:
print(len(counts))

44034


In [17]:
from scipy.sparse import lil_matrix
word_num = len(counts)//10
words = [x[1] for x in counts[:word_num]]
from nltk.corpus import stopwords
stopwords = stopwords.words("english")
filterword = [w for w in words if not w in stopwords]
#filterword = [w for w in words]
wordId = dict(zip(filterword, range(len(filterword))))
wordSet = set(filterword)
word_df = dict(zip(filterword, [0]*(len(filterword))))

for d in data1:
    r = ''.join([c for c in d['review_text'].lower() if not c in sp])
    tokens = r.split()
    sets= set(tokens)
    for i in sets:
        if i in filterword:
            word_df[i]+=1
print('done..')

done..


In [18]:
import copy
import math
word_df1= copy.deepcopy(word_df)
N=len(data1)
for i in word_df1:
    word_df1[i]= math.log(N/word_df1[i])
#print(word_df1)

In [20]:
def feature1(datum):
    feat = [0]*len(filterword)
    r = ''.join([c for c in datum['review_text'].lower() if not c in sp])
    tokens = r.split()
    for w in tokens:
        if w in wordSet:
            feat[wordId[w]] += 1
    sets=set(tokens)
    for s in sets:
        if s in wordSet:
            feat[wordId[s]]=feat[wordId[s]]*word_df1[s]

    feat.append(1)
    return feat

nf1 = len(feature1(data1[0]))

X1 = lil_matrix((len(data1), nf1))

for i in range(len(data1)):
    
    x = feature1(data1[i])
    for j in range(nf1):
        if x[j]:
            X1[i,j] = x[j]

print('done..')

done..


In [21]:
Xtrain1 = X1[:len(data1)//10*9]

Xtest1 = X1[len(data1)//10*9:]



mod_ifdif = linear_model.LogisticRegression(C=2.0,random_state=0)
mod_ifdif.fit(Xtrain1, Y_train)

pred_valid1= mod_ifdif.predict(Xtest1)
correct2 = pred_valid1==Y_test
print("accuracy feature:bag of words idf", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')
print('done')

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


accuracy feature:bag of words idf 0.7919640496960084
done..
done


In [22]:
dic1= defaultdict(int)
dic2=defaultdict(int)

#print(l[0])
for i in pred_valid1:                                       #num of fit/small/large
    j=str(i)
    dic1[j]+=1
for i in Y_test:                                       #num of fit/small/large
    j=str(i)
    dic2[j]+=1
    
print(dic1,dic2)

defaultdict(<class 'int'>, {'fit': 12811, 'large': 1152, 'small': 1169}) defaultdict(<class 'int'>, {'fit': 11163, 'small': 2005, 'large': 1964})


In [23]:
##i fdif result
dic1= defaultdict(int)
dic2=defaultdict(int)
dic3=defaultdict(int)
for i,j in zip(pred_valid1,Y_test):
    if(i==j and i =='small'):
        dic1[i]+=1
    elif (i==j and i =='large'):
        dic1[i]+=1
    elif(i==j and i =='fit'):
        dic1[i]+=1
    elif(i=='small'):
        dic1['predict small but wrong']+=1
    elif(i=='large'):
        dic1['predicted large but wrong']+=1
    else:
        dic1['predict fit but wrong']+=1
print(dic1)

defaultdict(<class 'int'>, {'fit': 10495, 'predict fit but wrong': 2316, 'predicted large but wrong': 392, 'small': 729, 'large': 760, 'predict small but wrong': 440})


In [52]:
def feature(datum):
    feat = [0]*len(filterword)
    r = ''.join([c for c in datum['review_text'].lower() if not c in sp])
    tokens = r.split()
    for w in tokens:
        if w in wordSet:
            feat[wordId[w]] += 1
    #feat.append(datum['height'])
    #feat.append(datum['weight'])
    feat.append(datum['rating'])
    feat.append(1)
    return feat

nf = len(feature(data1[0]))

X = lil_matrix((len(data1), nf))

for i in range(len(data1)):
    
    x = feature(data1[i])
    for j in range(nf):
        if x[j]:
            X[i,j] = x[j]

print('done..')

done..


In [53]:
#if(X!=X1):
#    print("false")

In [54]:
Xtrain = X[:len(data1)//10*9]

Xtest = X[len(data1)//10*9:]



mod1 = linear_model.LogisticRegression(C=2.0,random_state=0)
mod1.fit(Xtrain, Y_train)
print('done')

done


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [71]:
pred_valid= mod1.predict(Xtest)
correct3 = pred_valid==Y_test
#BER(pred_valid,Y_test)
print("accuracy feature:bag of words", sum(correct3)/len(correct3))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.7946074544012688
done..


In [70]:
FP=FalsePos(pred_valid,Y_test)
print(pred_valid,Y_test)

['fit' 'fit' 'fit' ... 'fit' 'fit' 'fit'] ['fit', 'fit', 'fit', 'fit', 'small', 'fit', 'large', 'fit', 'small', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'large', 'large', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'small', 'small', 'fit', 'fit', 'small', 'fit', 'fit', 'fit', 'fit', 'fit', 'large', 'fit', 'fit', 'fit', 'fit', 'fit', 'large', 'fit', 'large', 'fit', 'fit', 'fit', 'large', 'small', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'small', 'large', 'fit', 'small', 'fit', 'fit', 'large', 'fit', 'fit', 'fit', 'fit', 'small', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'large', 'fit', 'fit', 'fit', 'fit', 'large', 'large', 'fit', 'fit', 'fit', 'fit', 'large', 'fit', 'fit', 'fit', 'fit', 'large', 'fit', 'large', 'fit', 'fit', 'small', 'fit', 'small', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'large', 'small', 'large', 'fit', 'small', 'large', 'fit', 'fit', 'large', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'fit', 'large', 'l

In [56]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.7946074544012688
done..


In [57]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.7946074544012688
done..


In [58]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.7946074544012688
done..


In [59]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.7946074544012688
done..


In [60]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.7946074544012688
done..


In [61]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.7946074544012688
done..


In [62]:
pred_valid= mod1.predict(Xtest)
correct2 = pred_valid==Y_test
print("accuracy feature:bag of words", sum(correct2)/len(correct2))
#print(len(correct1))
print('done..')

accuracy feature:bag of words 0.7946074544012688
done..


In [63]:
dic1= defaultdict(int)
dic2=defaultdict(int)

#print(l[0])
for i in pred_valid:                                       #num of fit/small/large
    j=str(i)
    dic1[j]+=1
for i in Y_test:                                       #num of fit/small/large
    j=str(i)
    dic2[j]+=1
    
print(dic1,dic2)

defaultdict(<class 'int'>, {'fit': 13010, 'large': 1061, 'small': 1061}) defaultdict(<class 'int'>, {'fit': 11163, 'small': 2005, 'large': 1964})


In [64]:
dic1= defaultdict(int)
dic2=defaultdict(int)
dic3=defaultdict(int)
for i,j in zip(pred_valid,Y_test):
    if(i==j and i =='small'):
        dic1[i]+=1
    elif (i==j and i =='large'):
        dic1[i]+=1
    elif(i==j and i =='fit'):
        dic1[i]+=1
    elif(i=='small'):
        dic1['predict small but wrong']+=1
    elif(i=='large'):
        dic1['predicted large but wrong']+=1
    else:
        dic1['predict fit but wrong']+=1
    

In [65]:
print(dic1,dic2)

defaultdict(<class 'int'>, {'fit': 10623, 'predict fit but wrong': 2387, 'predicted large but wrong': 338, 'small': 678, 'predict small but wrong': 383, 'large': 723}) defaultdict(<class 'int'>, {})
